### Inci's Preliminary Dispersion Correction Code

In [57]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from g4beam import *
from scan import *

import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
import numpy as np
import pandas as pd
from tqdm import *
import pandas as pd
import pickle
import itertools
from tabulate import tabulate
import tempfile
import glob

In [58]:
# Make Sure g4bl is here
import os
os.environ["PATH"] += os.pathsep + "/home/incik/G4beamline-3.08/bin"
import shutil
print(shutil.which("g4bl"))

/home/incik/G4beamline-3.08/bin/g4bl


In [59]:
# Load data POST WEDGE as a Dataframe so that you can use Daniel Fu's functions
filename = "particles_after.txt"

# Skip the first two header lines that start with '#'
with open(filename) as f:
    # Read until the line containing column names
    for line in f:
        if line.startswith("#x "):
            columns = line.strip().lstrip("#").split()
            break

# Now load the data into a DataFrame
df = pd.read_csv(filename, comment="#", delim_whitespace=True, names=columns)

x_params, y_params, z_emit = calc_all_params(df) # _params are tuples of the form (emittance, beta, gamma, alpha, D, D')
print(f"D_x: {x_params[4]}, D'_x: {x_params[5]}, D_y: {y_params[4]}, D'_y: {y_params[5]}")

D_x: 0.007405947668631706, D'_x: -0.18883973986720437, D_y: 5.278092595079276e-05, D'_y: 0.012149181979370281


/tmp/ipykernel_16733/4034819515.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(filename, comment="#", delim_whitespace=True, names=columns)


In [60]:
"""
Optimize a 2-bend + central quadrupole cell so that
D_x = D'_x = D_y = D'_y = 0 at a virtual detector location.

Approach:
- For each trial set of magnet params, create a g4bl input that produces a single
multi-particle run with a momentum spread and writes an ASCII zntuple with columns:
    x xp y yp delta
- Parse the file and compute dispersions and slopes via covariance (no ± runs).
- Minimize cost = w_x*(D_x^2 + D'_x^2) + w_y*(D_y^2 + D'_y^2).
"""

# ---------------- USER CONFIG ----------------
G4BEAMLINE_CMD = "g4bl"
TEMPLATE_FILE = "Achromat.g4bl"
OUTPUT_DIR = "achromat_runs"
VD_FILENAME = "vd_achromat.txt"   # virtual detector writes this file (ascii)
N_PARTICLES = 5000             # increase for lower noise
E0 = 1e4                       # beam energy (GeV) - for a muon collider
# objective weights
w_x = 1.0
w_y = 1.0
# variable bounds and names (example): [B1, B2, G, L1, L2, x1_offset, x2_offset]
var_names = ['N_PARTICLES', 'Z1', 'B1', 'B2', 'G1', 'LB1', 'LB2', 'L1', 'LQ', 'L2', 'L3', 'VD_FILENAME']
bounds = [(1000,50000),(-1.5, 1.5), (-1.5, 1.5), (-100.0, 100.0), (0.05, 5.0), (0.05, 5.0), (0.05, 5.0), (0.05, 5.0), VD_FILENAME]
xvec = np.array([N_PARTICLES, 100.0, 1.0, 1.0, 50, 200.0, 200.0, 250.0, 230.0, 240.0, 250.0, VD_FILENAME])
os.makedirs(OUTPUT_DIR, exist_ok=True)

"""params = {
    "E": E0,
    "B1": 0.7, "B2": -0.7,
    "G": 50.0,
    "L1": 0.5, "Lq": 0.3, "L2": 0.5, "L3": 0.2,
    "VD_FILENAME": "vd_out.txt",
    "N_PARTICLES": 5000
}"""

'params = {\n    "E": E0,\n    "B1": 0.7, "B2": -0.7,\n    "G": 50.0,\n    "L1": 0.5, "Lq": 0.3, "L2": 0.5, "L3": 0.2,\n    "VD_FILENAME": "vd_out.txt",\n    "N_PARTICLES": 5000\n}'

In [61]:
# OK workflow wise -- we'd like to write on to this file in the correct places, and then run it w g4bl, and then optimize
def write_input_from_template(template_path, out_path, replacements):
    with open(template_path, 'r') as f:
        txt = f.read()
    try:
        txt = txt.format(**replacements)
    except KeyError as e:
        raise RuntimeError(f"Template substitution failed; missing placeholder: {e}")
    with open(out_path, 'w') as f:
        f.write(txt)

params = {name: val for name, val in zip(var_names, xvec)}
print(params)
write_input_from_template(TEMPLATE_FILE, f"/home/incik/Cooling_4D/AchromatTest/{OUTPUT_DIR}/run.g4bl", params)

{'N_PARTICLES': np.str_('5000'), 'Z1': np.str_('100.0'), 'B1': np.str_('1.0'), 'B2': np.str_('1.0'), 'G1': np.str_('50'), 'LB1': np.str_('200.0'), 'LB2': np.str_('200.0'), 'L1': np.str_('250.0'), 'LQ': np.str_('230.0'), 'L2': np.str_('240.0'), 'L3': np.str_('250.0'), 'VD_FILENAME': np.str_('vd_achromat.txt')}
